In [1]:
import openai
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, f1_score, confusion_matrix, classification_report

# Establecer la clave de la API de OpenAI
openai.api_key = "API_KEY"

# Seleccionar modelo  "gpt-3.5-turbo"
model = "gpt-3.5-turbo"

# Cargar el archivo de Excel en un DataFrame de pandas
data = pd.read_excel("datos/data.xlsx")

# Definir el texto del prompt para la clasificación
prompt = """
    Tendrás un rol de clasificador de comentarios de una publicación relacionada con la vacuna contra el VPH.
    Sólo debes responder con un valor numérico.
    No tienes permitido responder otra cosa que no sean números. Las clasificaciones son:

    0: El comentario tiene una postura contraria a la vacuna contra el VPH (antivacuna).
    1: El comentario tiene una postura a favor de la vacuna contra el VPH (provacuna).
    2: El comentario refleja una duda o dudas relacionadas con la vacuna contra el VPH.
    3: El comentario habla de cualquier otra cosa.

    Trata de interpretar las intenciones de las personas, ya que se trata de comentarios de Facebook.
    Si no puedes clasificar, tu respuesta debe ser "3".

    Ahora, clasifica el siguiente comentario, teniendo en cuenta que tu respuesta es solo un número:
"""


In [ ]:
# Parámetros de configuración
batch_size = 20  # Tamaño del lote de comentarios a procesar antes de guardar
output_file = "datos/data.xlsx"  # Nombre del archivo de salida
checkpoint_file = "checkpoint.txt"  # Nombre del archivo de checkpoint

# Variable para almacenar la posición actual en el bucle
current_index = 0
completed = False

while not completed:
    # Verificar si existe un archivo de checkpoint
    try:
        with open(checkpoint_file, 'r') as f:
            current_index = int(f.read())
        print("Se encontró un archivo de checkpoint. Continuando desde la posición:", current_index)
    except FileNotFoundError:
        print("No se encontró un archivo de checkpoint. Comenzando desde el principio.")

    # Crear una columna vacía para almacenar las respuestas si aún no existe
    if 'Clasificación_gpt_data_cruda' not in data.columns:
        data['Clasificación_gpt_data_cruda'] = ''

    # Iterar sobre cada comentario en el DataFrame
    for index, row in data.iterrows():
        # Verificar si se debe retomar desde el punto de reinicio guardado
        if index < current_index:
            continue

        comment = row['Comment']

        try:
            # Crear la solicitud de completado de chat
            completion = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": prompt},
                    {"role": "user", "content": comment}
                ],
                temperature=0.1
            )

            response = completion.choices[0].message.content.strip()

            # Verificar si la respuesta es un número
            if response.isdigit():
                # Convertir la respuesta a entero
                response = int(response)
            else:
                # Manejar el caso en el que la respuesta no es un número
                # Puedes asignar un valor predeterminado o tomar cualquier otra acción apropiada
                response = None  # o cualquier otro valor predeterminado que prefieras

            data.at[index, 'Clasificación_gpt_data_cruda'] = response

            # Guardar el DataFrame en un archivo después de procesar un lote de comentarios
            if (index + 1) % batch_size == 0 or (index + 1) == len(data):
                data[:index + 1].to_excel(output_file, index=False)
                print("Guardando...")

            # Guardar la posición actual como punto de reinicio
            with open(checkpoint_file, 'w') as file:
                file.write(str(index + 1))

        except openai.error.OpenAIError as e:
            # Manejar el error del servidor de OpenAI
            print("Error del servidor de OpenAI:", e)
            print("Reanude el proceso desde la iteración", index)
            completed = False
            break

    else:
        # El bucle for se completó sin errores, terminar el proceso
        completed = True

print("Procesamiento completado y resultados guardados en", output_file)


In [ ]:
# Cargar los datos procesados en un nuevo DataFrame
data = pd.read_excel("data_cruda_gpt.xlsx")

# Eliminar filas con valores faltantes en la columna de clasificación
data = data.dropna(subset=["Clasificación_gpt_data_cruda"])

# Obtener las etiquetas predichas y las etiquetas reales
y_pred = data["Clasificación_gpt_data_cruda"]
y_true = data["Topic_c"]

# Calcular la matriz de confusión
confusion = confusion_matrix(y_true, y_pred)
print("Matriz de Confusión:\n", confusion)

# Calcular la sensibilidad y especificidad para cada clase
for i in range(confusion.shape[0]):
    true_positive = confusion[i, i]
    false_negative = confusion.sum(axis=1)[i] - true_positive
    false_positive = confusion.sum(axis=0)[i] - true_positive
    true_negative = confusion.sum() - (true_positive + false_positive + false_negative)

    sensitivity = true_positive / (true_positive + false_negative)
    specificity = true_negative / (true_negative + false_positive)

    print("Valor:", i)
    print("Sensibilidad:", sensitivity)
    print("Especificidad:", specificity)
    print("Matriz de Confusión para el valor", i, ":\n", [[true_positive, false_positive], [false_negative, true_negative]])
    print()

# Generar el informe de clasificación
classification_report_text = classification_report(y_true, y_pred)
print("Informe de Clasificación:\n", classification_report_text)

# Guardar los resultados en un archivo separado
with open('resultados.txt', 'w') as file:
    file.write("Matriz de Confusión:\n")
    file.write(str(confusion))
    file.write("\n\n")

    for i in range(confusion.shape[0]):
        true_positive = confusion[i, i]
        false_negative = confusion.sum(axis=1)[i] - true_positive
        false_positive = confusion.sum(axis=0)[i] - true_positive
        true_negative = confusion.sum() - (true_positive + false_positive + false_negative)

        sensitivity = true_positive / (true_positive + false_negative)
        specificity = true_negative / (true_negative + false_positive)

        file.write("Valor: " + str(i) + "\n")
        file.write("Sensibilidad: " + str(sensitivity) + "\n")
        file.write("Especificidad: " + str(specificity) + "\n")
        file.write("Matriz de Confusión para el valor " + str(i) + ":\n")
        file.write(str([[true_positive, false_negative], [false_positive, true_negative]]))
        file.write("\n\n")

    file.write("Informe de Clasificación:\n")
    file.write(classification_report_text)

print("Los resultados se han guardado en el archivo 'resultados.txt'.")